In [3]:
import os
import shutil
import glob
from pathlib import Path
from google.colab import drive
from sklearn.model_selection import train_test_split # Needed in Block 2

# --- 1. CONFIGURATION: Verified Paths ---

# ⚠️ This is the FINAL CORRECTED ROOT PATH based on your diagnostic output.
DRIVE_DATA_ROOT = "/content/drive/MyDrive/weapon detection dataset for YOLOv5/dataset/dataset"

# Define the local temporary training path (DO NOT CHANGE)
BASE_DIR = "/content/YOLO_AntiPoaching_Train"
YAML_PATH = f"{BASE_DIR}/poaching_config.yaml"
print(f"Working Directory: {BASE_DIR}")


# --- 2. INSTALLATION AND MOUNT ---
drive.mount('/content/drive')
! pip install ultralytics # Installs YOLOv8

# Clean and create local folders
! rm -rf $BASE_DIR
! mkdir -p $BASE_DIR/images $BASE_DIR/labels

# --- 3. COPY AND REMAP DATA ---

# Source folders are relative to DRIVE_DATA_ROOT
IMAGES_SOURCE_DIR = f"{DRIVE_DATA_ROOT}/images"
LABELS_SOURCE_DIR = f"{DRIVE_DATA_ROOT}/labels"
print(f"\nDiscovered Image Source: {IMAGES_SOURCE_DIR}")
print(f"Discovered Label Source: {LABELS_SOURCE_DIR}")


# 1. Copy All Images (Recursively searches train/val/etc. subfolders)
print("\nStarting Image Copy to local storage...")
! cp -r "$IMAGES_SOURCE_DIR"/*/* $BASE_DIR/images/  # Copies all files from train/, val/ subfolders
print(f"Total images copied locally: {len(os.listdir(f'{BASE_DIR}/images'))}")


# 2. CRITICAL STEP: REMAP LABELS to ID 1 (Weapon_Tool) and copy locally
count_remapped_labels = 0
for root, _, filenames in os.walk(LABELS_SOURCE_DIR):
    for filename in filenames:
        if filename.endswith('.txt'):
            source_filepath = os.path.join(root, filename)
            destination_filepath = os.path.join(BASE_DIR, 'labels', filename)

            with open(source_filepath, 'r') as f:
                lines = f.readlines()

            with open(destination_filepath, 'w') as f:
                for line in lines:
                    parts = line.strip().split()
                    if parts:
                        # Unify all weapon-related classes (IDs 0, 1, etc.) to the new ID 1.
                        parts[0] = '1'
                        f.write(' '.join(parts) + '\n')
                        count_remapped_labels += 1

print(f"\nLabels Processed and Remapped to Class ID 1: {count_remapped_labels}")
print("--- Block 1 Execution Complete: Data Ready ---")

Working Directory: /content/YOLO_AntiPoaching_Train
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Discovered Image Source: /content/drive/MyDrive/weapon detection dataset for YOLOv5/dataset/dataset/images
Discovered Label Source: /content/drive/MyDrive/weapon detection dataset for YOLOv5/dataset/dataset/labels

Starting Image Copy to local storage...
Total images copied locally: 4156

Labels Processed and Remapped to Class ID 1: 4651
--- Block 1 Execution Complete: Data Ready ---


In [4]:
import glob
from sklearn.model_selection import train_test_split # Ensure this is installed
import os
import shutil
import numpy as np

# Define paths (re-using variables from Block 1)
BASE_DIR = "/content/YOLO_AntiPoaching_Train"
YAML_PATH = f"{BASE_DIR}/poaching_config.yaml"
print(f"Configuration file path: {YAML_PATH}")

# 1. Prepare for file splitting
all_image_paths = glob.glob(f"{BASE_DIR}/images/*.jpg") # Get all images for splitting

# Create split directories (train/val/test) inside the BASE_DIR
for split in ['train', 'val', 'test']:
    os.makedirs(f"{BASE_DIR}/{split}/images", exist_ok=True)
    os.makedirs(f"{BASE_DIR}/{split}/labels", exist_ok=True)


# 2. Perform the dataset split (70% Train / 20% Val / 10% Test)
print("\nSplitting dataset into train/val/test folders (70/20/10)...")

train_ratio, val_ratio = 0.7, 0.2
test_ratio = 1.0 - train_ratio - val_ratio

# Split filenames (without extensions)
base_names = [os.path.basename(p).split('.')[0] for p in all_image_paths]
train_names, test_names = train_test_split(base_names, test_size=test_ratio, random_state=42)
train_names, val_names = train_test_split(train_names, test_size=val_ratio/(train_ratio), random_state=42)

def move_split(file_names, split_name):
    """Moves the images AND their corresponding label files."""
    for name in file_names:
        # Move image
        shutil.move(f"{BASE_DIR}/images/{name}.jpg", f"{BASE_DIR}/{split_name}/images/{name}.jpg")
        # Move corresponding label
        shutil.move(f"{BASE_DIR}/labels/{name}.txt", f"{BASE_DIR}/{split_name}/labels/{name}.txt")

move_split(train_names, 'train')
move_split(val_names, 'val')
move_split(test_names, 'test')

print(f"Dataset split successful. Train: {len(train_names)}, Val: {len(val_names)}, Test: {len(test_names)}")


# 3. Define final YAML configuration content
yaml_content = f"""
path: {BASE_DIR}
train: train/images
val: val/images
test: test/images

nc: 3
names: ['Human', 'Weapon_Tool', 'Animal_Vehicle_Background']
"""
with open(YAML_PATH, 'w') as f:
    f.write(yaml_content)


# 4. Download YOLOv8m and Start Training
! wget -q https://github.com/ultralytics/assets/releases/download/v8.2.24/yolov8m.pt

print("\nStarting YOLOv8 Medium Fine-Tuning. Estimated completion: ~3-4 hours.")

! yolo task=detect mode=train \
    model=yolov8m.pt \
    data={YAML_PATH} \
    epochs=30 \
    imgsz=640 \
    batch=8 \
    name=AntiPoaching_QuickModel_Final

print("\n--- Block 2 Execution Complete: TRAINING LAUNCHED ---")
print("You must wait for the cell to finish (or time out) before the next step.")

Configuration file path: /content/YOLO_AntiPoaching_Train/poaching_config.yaml

Splitting dataset into train/val/test folders (70/20/10)...
Dataset split successful. Train: 2671, Val: 1069, Test: 416

Starting YOLOv8 Medium Fine-Tuning. Estimated completion: ~3-4 hours.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.209 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/YOLO_AntiPoaching_Train/poaching_config.yaml, degrees=

In [5]:
# 1. Define paths
TRAINED_MODEL_PATH="/content/runs/detect/AntiPoaching_QuickModel_Final/weights/best.pt"
ONNX_EXPORT_PATH="/content/runs/detect/AntiPoaching_QuickModel_Final/weights/best.onnx"
# ⚠️ UPDATE THIS DRIVE PATH to ensure you save the file correctly
DRIVE_SAVE_PATH="/content/drive/MyDrive/Final_AntiPoaching_Model_V1.onnx"

# 2. Export the model to ONNX format (Essential for fast backend deployment)
print("Starting ONNX Export...")
! yolo export model=$TRAINED_MODEL_PATH format=onnx

# 3. Copy the final ONNX model to Google Drive (for persistence)
! cp $ONNX_EXPORT_PATH "$DRIVE_SAVE_PATH"

print(f"\n--- Model Export and Save Complete ---")
print(f"Final Model saved permanently to Drive: {DRIVE_SAVE_PATH}")
print("You now have the core AI asset for deployment.")

Starting ONNX Export...
Ultralytics 8.3.209 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 92 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from '/content/runs/detect/AntiPoaching_QuickModel_Final/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (49.6 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 2.3s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.71...
ONNX: export success ✅ 5.8s, saved as '/content/runs/detect/AntiPoaching_QuickModel_Final/weights/best.onnx' (98.8 MB)

Export complete (8.3s)
Res